<h1><center>Part 3: Topics Modeling </center></h1>

In part 3, we use the result found in part 1 and apply Latent Dirichlet Allocation (LDA) model to identify different topics discussed by the reviewers. The data is split into two parts:
1. Positive reviewes, we will find the main topics in this reviews.
2. Negative reviewes, we will find the main topics in this reviews.

Topic modeling is an unsupervised learning technique used to represent text documents with the help of several topics. It does not require a predefined list of tags for documents, instead, it analyzes text data to determine cluster words(topics) for a set of documents


## 1. Data Preparation 

#### Import of different libraries that will be used in this part of the project.

In [1]:
import pandas as pd
import unicodedata
import re
import math

#Gensim
import nltk
import contractions
import string
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#spacy
import spacy
from nltk.corpus import stopwords
#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [2]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nordine.quadar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nordine.quadar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# read already saved data
dat = pd.read_csv('asin_sample_GPT_Rev.csv')

In [4]:
# Checking the data 
dat

,Unnamed: 0,overall,vote,verified,reviewerID,asin,reviewText,summary,unixReviewTime,reviewText_clean,reviewText_deep_clean,year,month,Sentiment_Score,Sentiment_GPT_Score,Reviewer_Type
0,0,5,2,No,AFDH6LFI9LP4E,B00KJ07SEM,Im not buying the GE one again. This one works...,Best Water Filter Ever!,2014-07-08,im not buying the ge one again. this one works...,im buy ge one thi one work great much less cos...,2014,7,5,0,-1
1,1,5,2,Yes,A1GZT67WOLNL5F,B00KJ07SEM,Removed the GE MWF Smartwater filter inserted ...,Worked like a charm,2014-07-06,removed the ge mwf smartwater filter inserted ...,remov ge mwf smartwat filter insert waterfal f...,2014,7,5,1,0
2,2,4,2,Yes,A3CMR3EQ6NSYEE,B00KJ07SEM,This a good filter and fits our needs quite we...,Good non OEM filter,2014-06-30,this a good filter and fits our needs quite we...,thi good filter fit need quit well work well g...,2014,6,5,1,1
3,3,3,5,No,A2ZHDU2DP6VYU9,B00KJ07SEM,Update: Within hours of my posting this review...,"Update: Terrible filter, but great customer se...",2014-06-27,update: within hours of my posting this review...,updat within hour post review first time jenni...,2014,6,3,0,0
4,4,5,30,Yes,A1GI72ZP1HD0V8,B00KJ07SEM,I have a GE two door refrigerator and use to b...,Very good,2014-06-22,i have a ge two door refrigerator and use to b...,i ge two door refriger use buy origin filter c...,2014,6,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194,3195,5,0,Yes,A1FK4897PKMVSJ,B00KJ07SEM,"Snapped in no problem, filters ok, lasted abou...",Five Stars,2018-07-22,"snapped in no problem, filters ok, lasted abou...",snap problem filter ok last year,2018,7,4,0,0
3195,3196,5,0,Yes,AA5HIR6MSWTIL,B00KJ07SEM,I have ordered this filter from this seller mu...,Fridge filter,2018-07-22,i have ordered this filter from this seller mu...,i order filter seller multipl time alway posit...,2018,7,5,1,1
3196,3197,5,0,Yes,A35NA14R0332AZ,B00KJ07SEM,Why spend $400 more a year for some name brand...,"Buy this now, but read funny reveiw",2018-07-17,why spend $400 more a year for some name brand...,whi spend 400 year name brand crap these work ...,2018,7,1,-1,-1
3197,3198,5,0,Yes,A16N4OCSB7J50F,B00KJ07SEM,Filter works great so much so I just reordered...,Five Stars,2018-07-14,filter works great so much so i just reordered...,filter work great much i reorder backup,2018,7,5,1,1


After pulling and refining the data from part 1 and importing the required packages, we need to clean the data from NaN and urls so we can tokenize it for the next steps.

In [5]:
# Remove 'nan' values from cleaned reveiw 
for j in dat['reviewText_clean'].index:
    x = dat['reviewText_clean'][j]
    if isinstance(x, (int, float)) and math.isnan(x):
        dat = dat.drop(index=j)
        print(j)

In [6]:
# standardizing accented characters if any
def standardize_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

#standardizing accented characters for every row
dat.reviewText_clean = dat.reviewText_clean.apply(standardize_accented_chars)

In [7]:
def get_number_of_urls(documents):
    print("{:.2f}% of documents contain urls".format(sum
(documents.apply(lambda x:x.find('http'))>0)/len
(documents)*100))
    
# Passing the 'reviewText_clean' column of the dataframe as the argument
print(get_number_of_urls(dat.reviewText_clean))

0.13% of documents contain urls
None


In [8]:
def remove_url(text):
     return re.sub(r'https?:\S*', '', text)

#removing urls from every row
dat.reviewText_clean=dat.reviewText_clean.apply(remove_url)

In [9]:
def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
           expanded_words.append(contractions.fix(word)) 
    return ' ' .join(expanded_words)

#expanding contractions for every row
dat.reviewText_clean=dat.reviewText_clean.apply(expand_contractions)

**Expanding Contractions:** Contractions are shortened versions of words. They are created by removing, one or more letters from words. More often than not, multiple words are combined to create a contraction. For example, I will is contracted into I’ll, do not into don’t. We wouldn’t want our model to consider I will and I’ll differently. Hence, we will convert each contraction into its expanded form using the below-mentioned code.

In [10]:
def keep_only_alphabet(text):
    return re.sub(r'[^a-z]', ' ', text)

#for all the rows
dat.reviewText_clean=dat.reviewText_clean.apply(keep_only_alphabet)

**Keeping only Alphabet:** Punctuations, numbers, and special characters like ‘$, %, etc.’ don’t provide any information. Hence, we will keep only letters and remove everything else present in the text using the below-mentioned function.

In [11]:
def remove_stopwords(text,nlp,custom_stop_words=None, remove_small_tokens=True,min_len=2):
    # if custom stop words are provided, then add them to default stop words list
    if custom_stop_words:
        nlp.Defaults.stop_words |= custom_stop_words
    
    filtered_sentence =[] 
    doc=nlp(text)
    for token in doc:
        if token.is_stop == False: 
            
            # if small tokens have to be removed, then select only those which are longer than the min_len 
            if remove_small_tokens:
                if len(token.text)>min_len:
                    filtered_sentence.append(token.text)
            else:
                filtered_sentence.append(token.text)
    # if after the stop word removal, words are still left in the sentence, then return the sentence as a string else return null 
    return ' '.join(filtered_sentence) if len(filtered_sentence)>0 else None
#creating a spaCy object. 
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
#removing stop-words and short words from every row
dat.reviewText_clean=dat.reviewText_clean.apply(lambda x:remove_stopwords(x,nlp))

In [12]:
# Remove 'nan' values from cleaned reveiw 
for j in dat['reviewText_clean'].index:
    x = dat['reviewText_clean'][j]
    if type(x) != str:
        dat = dat.drop(index=j)

In [13]:
def lemmatize(text, nlp):
    
    doc = nlp(text)
    lemmatized_text = []
    for token in doc:
        lemmatized_text.append(token.lemma_)
    return ' '.join(lemmatized_text)

#Performing lemmatization on every row
dat.reviewText_clean=dat.reviewText_clean.apply(lambda x:lemmatize(x,nlp))

**Lemmatization:** Lemmatization generates the root of the word. It makes use of vocabulary and morphological analysis of words, to generate the root form of a word. We will use the spaCy library for performing lemmatization.

In [14]:
def generate_tokens(review):
    words=[]
    for word in review.split(' '):
    # using the if condition because we introduced extra spaces during text cleaning
        if word!=' ':
            words.append(word)
    return words

#storing the generated tokens in a new column named 'words'
dat['tokens']=dat.reviewText_clean.apply(generate_tokens)

In [15]:
# Remove empty element from tokens
for elem in dat['tokens']:
    for subelem in elem:
        if '' in elem:
            elem.remove('')

## 2. Topics for Positive sentiment

In [16]:
# Select just the postive reviews to anlyze
dat_pos = dat[dat['Sentiment_GPT_Score']==1]

**Generating Document Matrix and Dictionary:**


The LDA topic model algorithm requires a document word matrix and a dictionary as the main inputs.

A document-term matrix is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.

A Dictionary is collection of all unique tokens present in documents.

For generating the document-term matrix and dictionary, first we will convert the reviews into tokens using the fucntion mentioned below.

In [17]:
def create_dictionary(words):
    return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word=create_dictionary(dat_pos.tokens)
print(id2word)

Dictionary(1608 unique tokens: ['charm', 'filter', 'insert', 'instruction', 'like']...)


In [18]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
        corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(dat_pos.tokens,id2word)

**LDA Implementing:**

Gensim library is used for LDA. For generating the base model, we use num_topics=5.

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
 id2word=id2word,
 num_topics=5,
 random_state=100,
 )

**LDA Topics Generation:**

We iterate over the topics identified by our LDA model, get the top words in each topic. We will store the top 10 words for each topic in a dataframe using the below-mentioned function. The top_n_words parameter controls the number of top words we want to store for each topic

In [20]:
def get_lda_topics(model, num_topics, top_n_words):
    word_dict = {}
    for i in range(num_topics):
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
 
    return pd.DataFrame(word_dict)
                  
get_lda_topics(lda_model,5,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,filter,good,work,filter,filter
1,great,filter,great,water,great
2,water,water,easy,taste,water
3,work,product,install,great,product
4,fit,easy,price,work,taste
5,easy,price,good,price,work
6,install,work,brand,good,buy
7,price,time,product,refrigerator,price
8,good,install,original,fit,service
9,product,arrive,cheap,easy,order


**Visualizing Topics:**

We use the pyLDAvis library for visualizing the results. The below-mentioned code will generate a dashboard that displays the results.

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
vis

C:\Users\nordine.quadar\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.064356 -0.071836       1        1  33.634290
4      0.005588  0.067263       2        1  20.596735
0     -0.036415 -0.071210       3        1  19.591396
2     -0.139346  0.029786       4        1  15.791504
1      0.105816  0.045996       5        1  10.386075, topic_info=       Term        Freq       Total Category  logprob  loglift
9      work  900.000000  900.000000  Default  20.0000  20.0000
19    great  954.000000  954.000000  Default  19.0000  19.0000
14     good  439.000000  439.000000  Default  18.0000  18.0000
32     easy  411.000000  411.000000  Default  17.0000  17.0000
65  install  345.000000  345.000000  Default  16.0000  16.0000
..      ...         ...         ...      ...      ...      ...
65  install   32.392901  345.724043   Topic5  -4.2446  -0.1030
13      fit   25.701331  255.441467   Topic5  -4.4760  -0.0317
9      work   37.642767  900.060746   Topic5  -4.0944  -0.9096
25    taste   28.432190  451.365534   Topic5  -4.3750  -0.5001
16      buy   23.390432  222.836189   Topic5  -4.5702   0.0106

[268 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
349       2  0.523013      
349       3  0.136438      
349       4  0.318356      
452       1  0.908180  able
452       4  0.100909  able
...     ...       ...   ...
63        1  0.665434  year
63        2  0.160622  year
63        3  0.068838  year
63        4  0.045892  year
63        5  0.068838  year

[543 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 3, 2])

**Comment:** We can see from this identified topics of the positive reviews that customers are mainly mentionning, in topic 1, that product is easy to fit and cheap and work great. We can see different features of the rpoduct for these topics.

## 3. Topics for Negative sentiment

We do the same for negative reviews

In [22]:
dat_neg = dat[dat['Sentiment_GPT_Score']== -1]

In [23]:
def create_dictionary(words):
    return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word=create_dictionary(dat_neg.tokens)
print(id2word)

Dictionary(1171 unique tokens: ['company', 'compatible', 'drain', 'forward', 'go']...)


In [24]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
        corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(dat_neg.tokens,id2word)

In [25]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
 id2word=id2word,
 num_topics=5,
 random_state=100,
 )

In [26]:
def get_lda_topics(model, num_topics, top_n_words):
    word_dict = {}
    for i in range(num_topics):
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
 
    return pd.DataFrame(word_dict)
                  
get_lda_topics(lda_model,5,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,filter,filter,taste,filter,filter
1,water,water,water,water,water
2,taste,buy,filter,taste,taste
3,work,taste,like,buy,work
4,fit,work,month,like,like
5,flow,product,work,work,product
6,mwf,instal,product,good,save
7,refrigerator,refrigerator,bad,fridge,purchase
8,old,brand,plastic,bad,get
9,month,bad,buy,return,return


In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

C:\Users\nordine.quadar\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.018321  0.008066       1        1  38.308134
3     -0.019121 -0.060651       2        1  29.191161
0     -0.075980 -0.007569       3        1  19.219173
4      0.016337  0.069519       4        1   8.710112
2      0.097085 -0.009365       5        1   4.571420, topic_info=        Term        Freq       Total Category  logprob  loglift
33     taste  178.000000  178.000000  Default  30.0000  30.0000
37     water  393.000000  393.000000  Default  29.0000  29.0000
28      like   62.000000   62.000000  Default  28.0000  28.0000
9    product   61.000000   61.000000  Default  27.0000  27.0000
15      work   97.000000   97.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
4         go    1.417263   31.747481   Topic5  -5.3185  -0.0237
214    brand    1.367952   49.580294   Topic5  -5.3539  -0.5049
73     order    1.235349   22.969219   Topic5  -5.4559   0.1625
126     year    1.225168   19.401001   Topic5  -5.4642   0.3231
298  quality    1.201181   19.171592   Topic5  -5.4839   0.3152

[377 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
666       1  0.123822        
666       2  0.742929        
666       4  0.123822        
396       1  0.586668    able
396       3  0.293334    able
...     ...       ...     ...
126       5  0.051544    year
523       2  0.581381  yellow
523       3  0.290690  yellow
267       1  0.441984    yuck
267       4  0.441984    yuck

[622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 3])

**Comment:** We can see from this identified topics of the negative reviews that customers are mainly mentioning, like in topic 1, something about the taste of water. 

**Conclusion:**
AS we can see these topics modeling can help to understand what customers are discussing. It's still not clear. The words are repeated in different topics. This issue can be solved by performing hyper-parameter tuning.


### -----------------------------------------------------------------------------------------------------------------------------------------------------
### End Part 3